In [1]:
%loadFromPOM pom.xml

In [2]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [3]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [4]:
// find schedule with service type = General Practice on 2018-09-16
Bundle results = client.search()    
    .forResource(Schedule.class)
    .where(Schedule.SERVICE_TYPE.exactly().code("124"))
    //with date on 2018-09-16
    .where(Schedule.DATE.exactly().day("2018-09-16"))
    .returnBundle(Bundle.class)
    .execute();

//列印出所有的schedule所包含的practitioner
for (Bundle.BundleEntryComponent entry : results.getEntry()) {
    Schedule schedule = (Schedule) entry.getResource();
    System.out.println("Schedule ID: " + schedule.getId());
    System.out.println("Schedule Service Type: " + schedule.getServiceType().get(0).getText());
    System.out.println("Schedule Actor: " + schedule.getActor().get(0).getDisplay());
    System.out.println("Schedule Actor Reference: " + schedule.getActor().get(0).getReference());
}
Schedule schedule = (Schedule) results.getEntry().get(0).getResource();
System.out.println("Schedule ID: " + schedule.getId());

Schedule ID: http://localhost:8080/fhir/Schedule/109/_history/1
Schedule Service Type: null
Schedule Actor: Kelly Smith
Schedule Actor Reference: Practitioner/103
Schedule ID: http://localhost:8080/fhir/Schedule/109/_history/1


In [5]:
// Now let's search for slots defined for the schedules.
// We'll search for slots that are free on 2018-09-16
results = client.search()
    .forResource(Slot.class)
    .where(Slot.SCHEDULE.hasId("Schedule/" + schedule.getIdElement().getIdPart()))
    .where(Slot.STATUS.exactly().code("free"))
    .where(Slot.START.exactly().day("2018-09-16"))
    .returnBundle(Bundle.class)
    .execute();

//列印出所有的slot所包含的practitioner
for (Bundle.BundleEntryComponent entry : results.getEntry()) {
    Slot slot = (Slot) entry.getResource();
    System.out.println("Slot : " + slot.getStart() + " ~ " + slot.getEnd());
}

Slot : Sun Sep 16 15:00:00 CST 2018 ~ Sun Sep 16 16:00:00 CST 2018
Slot : Sun Sep 16 09:00:00 CST 2018 ~ Sun Sep 16 10:00:00 CST 2018
Slot : Sun Sep 16 12:00:00 CST 2018 ~ Sun Sep 16 13:00:00 CST 2018
Slot : Sun Sep 16 17:00:00 CST 2018 ~ Sun Sep 16 18:00:00 CST 2018
Slot : Sun Sep 16 14:00:00 CST 2018 ~ Sun Sep 16 15:00:00 CST 2018
Slot : Sun Sep 16 16:00:00 CST 2018 ~ Sun Sep 16 17:00:00 CST 2018
